In [2]:
using TSML
using TSML.Utils
using TSML.TSMLTypes
using TSML.TSMLTransformers
using TSML.Statifiers
using TSML.DataReaders
using DataFrames
using Dates
using Random

## Introduce some missing data

In [17]:
Random.seed!(123)
dt=[missing;rand(1:10,3);missing;missing;missing;rand(1:5,3)]
dat = DataFrame(Date= DateTime(2017,12,31,1):Dates.Hour(1):DateTime(2017,12,31,10) |> collect,Value = dt)

,Date,Value
,DateTime,Int64⍰
1,2017-12-31T01:00:00,missing
2,2017-12-31T02:00:00,5
3,2017-12-31T03:00:00,3
4,2017-12-31T04:00:00,2
5,2017-12-31T05:00:00,missing
6,2017-12-31T06:00:00,missing
7,2017-12-31T07:00:00,missing
8,2017-12-31T08:00:00,2
9,2017-12-31T09:00:00,1


## Get statistics including blocks of missing data

In [22]:
statfier = Statifier(Dict(:processmissing=>true))
fit!(statfier,dat)
res=transform!(statfier,dat)

,median,mean,q25,q75,kurtosis,skewness,variation,entropy,autocor,pacf,bmedian,bmean,bq25,bq75,bmiss,bmin,bmax
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64
1,2.5,3.0,2.0,4.5,-1.46939,0.280566,0.557773,-22.1628,0.520008,1.67811,2.0,2.0,1.5,2.5,0,1.0,3.0


## Try real data; aggregate by hour

In [14]:
fname = joinpath(dirname(pathof(TSML)),"../data/testdata.csv")
csvfilter = DataReader(Dict(:filename=>fname,:dateformat=>"dd/mm/yyyy HH:MM"))
valgator = DateValgator(Dict(:dateinterval=>Dates.Hour(1)))
valnner = DateValNNer(Dict(:dateinterval=>Dates.Hour(1)))
stfier = Statifier(Dict(:processmissing=>true))

Statifier(nothing, Dict(:processmissing=>true))

## Get statistics including blocks of missing data

In [19]:
mpipeline1 = Pipeline(Dict(
  :transformers => [csvfilter,valgator,stfier]
 )
)
fit!(mpipeline1)
respipe1 = transform!(mpipeline1)

,median,mean,q25,q75,kurtosis,skewness,variation,entropy,autocor,pacf,bmedian,bmean,bq25,bq75,bmiss,bmin,bmax
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64
1,10.35,11.557,10.0,12.3,0.730635,1.41283,0.200055,-1.09145e5,4.39315,1.04766,5.0,10.5589,3.0,6.0,0,1.0,2380.0


## Try imputing and get statistics

In [21]:
mpipeline2 = Pipeline(Dict(
  :transformers => [csvfilter,valgator,valnner,stfier]
 )
)
fit!(mpipeline2)
respipe2 = transform!(mpipeline2)

,median,mean,q25,q75,kurtosis,skewness,variation,entropy,autocor,pacf,bmedian,bmean,bq25,bq75,bmiss,bmin,bmax
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64
1,10.0,11.1362,10.0,11.5,2.37274,1.87452,0.187997,-2.36714e5,4.47886,1.06997,NaN,NaN,NaN,NaN,0,NaN,NaN
